In [1]:
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
env = GridworldEnv()

In [3]:
env.action_space, env.nS, env.nA, env.P[14], env.P[0][0], env.P[0][0][0], env.P[1][0][0][1]


(Discrete(4),
 16,
 4,
 {0: [(1.0, 10, -1.0, False)],
  1: [(1.0, 15, -1.0, True)],
  2: [(1.0, 14, -1.0, False)],
  3: [(1.0, 13, -1.0, False)]},
 [(1.0, 0, 0.0, True)],
 (1.0, 0, 0.0, True),
 1)

### Policy Evaluation
Policy evaluation uses the Bellman Expectation Equation:

\begin{align*} v_{k+1}(s) &= \mathbb{E}_\pi [R_{t+1} + \gamma v_k(S_{t+1}) \mid S_t=s] \\ 
&= \sum_a \pi(a|s) \sum_{s'}P_{ss'}^a [r + \gamma v_k(s')]
\end{align*}


In [4]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)
    while True:
        # TODO: Implement!
        delta = 0
        for s in range(env.nS):
            v = 0
            for a, a_prob in enumerate(policy[s]): 
                for prob, next_state, reward, done in env.P[s][a]: # each action has only 1 next state with prob 100%
                    v += a_prob * prob * (reward + discount_factor*V[next_state])
            delta = max(delta, abs(v-V[s]))
            V[s] = v
        if delta < theta:
            break
    return np.array(V)

In [5]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)
v

array([  0.        , -13.99993529, -19.99990698, -21.99989761,
       -13.99993529, -17.9999206 , -19.99991379, -19.99991477,
       -19.99990698, -19.99991379, -17.99992725, -13.99994569,
       -21.99989761, -19.99991477, -13.99994569,   0.        ])

In [6]:
# Test: Make sure the evaluated policy is what we expected
expected_v = np.array([0, -14, -20, -22, -14, -18, -20, -20, -20, -20, -18, -14, -22, -20, -14, 0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)